In [181]:
import re
import pickle
import time

import requests
from bs4 import BeautifulSoup

import pandas as pd

In [148]:
resp_elements = requests.get('https://en.wikipedia.org/wiki/Category:Lists_of_isotopes_by_element')
resp_elements.raise_for_status()
elements_html = BeautifulSoup(resp_elements.text, 'lxml')

In [166]:
link_list = elements_html.findAll('a')
link_list = ['http://www.wikipedia.org'+a.get('href') 
 for a in link_list 
 if (a.get('href') is not None and 'Isotopes_of_' in a.get('href'))]

In [177]:
exclude_citation = re.compile('\[.*\]')

def extract_data_row(row):
    line_data = []
    for entry in row.findAll('td'):
        inner_data = exclude_citation.sub('', sort_key_remover.sub('', entry.get_text()))
        line_data.append(inner_data)
        if entry.get('colspan') is not None:
            line_data.extend(['NA']*(int(entry.get('colspan')) -1) )
    return line_data 

def extract_text_from_non_hidden(node):
    return ''.join(i.text for i in node.findAll({'class': lambda x: x!= 'sortkey'}))

def get_table_from_links(link_address):
    resp = requests.get(link_address)
    resp.raise_for_status()
    html = BeautifulSoup(resp.text, 'lxml')
    table_list = html.findAll('table', {'class': 'wikitable'})
    isotope_table = None
    for table in table_list:
        if 'spin' in table.text:
            isotope_table = table
    if isotope_table is None:
        return None, resp
    rows = list(isotope_table.findAll('tr'))
    names = [exclude_citation.sub('', i.text.replace('\n', ' ')) for i in rows[0].findAll('th')]
    frame = pd.DataFrame([extract_data_row(i) for i in rows[1:]], columns=names)
    return frame, resp

In [178]:
a, b = get_table_from_links('https://en.wikipedia.org/wiki/Isotopes_of_uranium')

In [190]:
element_name = re.compile('of_(.*)$')
names_and_links = {element_name.findall(i)[0]: i for i in link_list}

In [ ]:
#element_data = {}

In [198]:

for i in names_and_links:
    if i not in element_data and i != 'holmium' :
        print(i)
        element_data[i] = get_table_from_links(names_and_links[i]) 
        time.sleep(10)

curium
platinum
iron
silver
helium
gadolinium
neutronium
zirconium
selenium
cadmium
ytterbium
radium
europium
bohrium
technetium
dysprosium
cobalt
chromium
zinc
boron
titanium
neodymium
fluorine
seaborgium
bismuth
potassium
oxygen
strontium
sulfur
astatine
lithium
gallium
phosphorus
uranium
erbium
beryllium
polonium
actinium
tungsten
einsteinium
fermium
mendelevium
silicon
nihonium
roentgenium
caesium
manganese
germanium
scandium
thulium
rhodium
neon
osmium
plutonium
tellurium
indium
meitnerium
unbinilium
samarium
hydrogen
rubidium
calcium
americium
francium
flerovium
bromine
vanadium
radon
berkelium
tennessine
neptunium
thallium
niobium
nobelium
yttrium
carbon
ununennium
gold
iridium
praseodymium
copper
tin
molybdenum
protactinium
barium
xenon
oganesson
livermorium
nickel
thorium
cerium
rutherfordium
aluminium
darmstadtium
lutetium
iodine
moscovium
palladium
promethium
dubnium
mercury
hassium
antimony
krypton
nitrogen
californium
chlorine
sodium
ruthenium
copernicium
magnesium
tantalu

In [199]:
set(names_and_links.keys()) - set(element_data.keys())

{'holmium'}

In [200]:
holmium_html = BeautifulSoup(requests.get('http://www.wikipedia.org/wiki/Isotopes_of_holmium').text, 'lxml') 

In [206]:
t = holmium_html.findAll('table', {'class': 'wikitable'})[0]

In [220]:
rows = list(t.findAll('tr'))
names = [exclude_citation.sub('', i.text.replace('\n', ' ')) for i in rows[0].findAll('th')]
frame = pd.DataFrame([extract_data_row(i) for i in rows[1:] if len(extract_data_row(i)) == 10], columns=names)

In [222]:
element_data['holmium'] = (frame, requests.get('http://www.wikipedia.org/wiki/Isotopes_of_holmium') )

In [224]:
?pickle.dump

In [227]:
with open('element_data.pickle', 'wb') as f:
    pickle.dump(element_data, f)